In [1]:
from deta import Deta
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
from yahoo_fin.stock_info import *
import plotly.graph_objects as go

In [2]:
class StockyDb:
    def __init__(self):
        self.deta = Deta('d0p5if1f_GSnmoPk32YPhwKaJzN6sq7hM2DN4XPks')
        self.db = self.deta.Base('StockyAI')
        
    def save_to_DB(self,ticker,df):
        date_=[]
        open_=[]
        high_=[]
        low_=[]
        close_=[]
        for i in range (0,len(df)):
            df['Close']=df['Close'].astype(float)
            df['Low']=df['Low'].astype(float)
            df['High']=df['High'].astype(float)
            df['Open']=df['Open'].astype(float)
            date_.append(df['Date'][i])
            open_.append(df['Open'][i])
            high_.append(df['High'][i])
            low_.append(df['Low'][i])
            close_.append(df['Close'][i])
        self.db.put({'key':ticker,'date':date_,'open':open_,'high':high_,'low':low_,'close':close_})
    
    def ticker_df(self,ticker):
        return self.db.get(key=ticker)
    
    def get_all_ticker(self):
        tickers=[]
        for i in range(0,self.db.fetch().count):
            tickers.append(self.db.fetch().items[i]['key'])
        return tickers
    
    def Recomodation(self):
    tickers = DB.get_all_ticker()
    buy=[]
    sell = []
    
    for i in tickers:
        rec={}
        df=pd.DataFrame(DB.ticker_df(i))
        H_max=df['high'].max()
        L_min=df['low'].min()
        H_date=(df[df['high']==H_max])['date'].to_list()
        L_date=(df[df['low']==L_min])['date'].to_list()
        if H_date>L_date:
            price={}
            price['Ticker']=i
            price['L_buy_price']=L_min
            price['H_buy_price']=L_min+(L_min*0.01)
            price['H_sell_price']=H_max
            price['L_sell_price']=H_max-(H_max*0.01)
            price['profit% ']=((price['L_sell_price']-price['H_buy_price'])/price['H_buy_price'])*100
            buy.append(price)
        elif H_date<L_date:
            price={}
            price['Ticker']=i
            price['H_sell_price']=H_max
            price['L_sell_price']=H_max-(H_max*0.01)
            price['L_buy_price']=L_min
            price['H_buy_price']=L_min+(L_min*0.01)
            price['profit% ']=((price['L_sell_price']-price['H_buy_price'])/price['H_buy_price'])*100
            sell.append(price)
    
    buy_df=pd.DataFrame()
    for i in range (0,len(buy)):
        buy_df=buy_df.append([buy[i]],ignore_index=True)
    sell_df=pd.DataFrame()
    for j in range (0,len(sell)):
        sell_df=sell_df.append([sell[j]],ignore_index=True)
    return buy_df,sell_df

IndentationError: expected an indented block (Temp/ipykernel_11884/876515894.py, line 34)

In [161]:
dbp = deta.Base('StockyAI_portfolio')

In [2]:
class Portfolio():
    def __init__(self):
        self.dbp = deta.Base('StockyAI_portfolio')
        self.p_data = self.dbp.get(key='praveen')
        self.cash = self.p_data['cash']
        self.stocks = self.p_data['stocks']
        #self.S_df = pd.DataFrame(columns=['stock','buy_price','quantity'])
        #self.S_df=self.S_df.append(self.stocks,ignore_index=True)
        print(self.stocks)
        
    def Buy (self,ticker):
        tik= ticker
        ticker = yf.Ticker(ticker)
        price = ticker.info
        if (price['regularMarketPrice']) == None :
            pre_day_price=ticker.history(period='1d')
            print(ticker,"Price - ",pre_day_price.iloc[0,3])
            c_price =pre_day_price.iloc[0,3]
            quant= int(input('Quantity'))
            amt = c_price*quant
            if self.cash >= amt:
                self.cash=self.cash-amt
                S_detail={}
                S_detail['buy_price']=c_price
                S_detail['quantity'] = quant
                self.stocks[tik]=S_detail
                #return self.stocks
                self.dbp.put({'key':'praveen','cash':self.cash,'stocks':self.stocks})
                
            else:
                print('INSUFICENT BALANCE')

        else:
            print("Price - ",price['regularMarketPrice'])
            c_price =price['regularMarketPrice']
            quant= int(input('Quantity'))
            amt = c_price*quant
            if self.cash >= amt:
                self.cash=self.cash-amt
                print('stocks brought')
                print('available balance - ',cash)
            else:
                print('INSUFICENT BALANCE')

    def Sell (self,ticker):
        tik=ticker
        ticker = yf.Ticker(ticker)
        price = ticker.info
        if (price['regularMarketPrice']) == None :
            pre_day_price=ticker.history(period='1d')
            print(ticker,"Price - ",pre_day_price.iloc[0,3])
            c_price =pre_day_price.iloc[0,3]
            quant= int(input('Quantity'))
            amt = c_price*quant
            if self.stocks[tik]['quantity']>= quant :
                self.cash=self.cash+amt
                r_quant = self.stocks[tik]['quantity']-quant
                S_details=(self.stocks[tik])
                S_details['quantity']=r_quant
                self.stocks[tik]=S_details
                #print(self.stocks[tik])
                #return self.stocks
                self.dbp.put({'key':'praveen','cash':self.cash,'stocks':self.stocks})
                
                
            else:
                print('INSUFICENT BALANCE')

        else:
            print("Price - ",price['regularMarketPrice'])
            c_price =price['regularMarketPrice']
            quant= int(input('Quantity'))
            amt = c_price*quant
            if cash >= 0:
                cash=cash+amt
                print('stocks Sold')
                print('available balance - ',cash)
            else:
                print('INSUFICENT BALANCE')
        
    def get_holdings(self):
        #print(self.p_data['stocks'])
        hold_df = pd.DataFrame(self.p_data['stocks'])
        hold_df=hold_df.transpose()
        hold_df=hold_df.reset_index()
        hold_df['index']=hold_df['index'].astype('str')
        tiks=hold_df['index'].to_list()
        #tickers=yf.Tickers(tiks)
        prices=[]
        
        for i in (tiks):
            #i = tickers.tickers[i]
            pr = get_live_price(i)
            #pr = pr['regularMarketPrice']
            prices.append(pr)
            
        #hold_df['current price']=hold_df['index'].apply(self.get_current_price)
        hold_df['current price']=prices
        return hold_df
    
    def get_current_price(self,ticker):
        ticker=yf.Ticker(ticker)
        info=ticker.info
        return info['regularMarketPrice']

In [3]:
deta = Deta('d0p5if1f_GSnmoPk32YPhwKaJzN6sq7hM2DN4XPks')
dbh = deta.Base('StockyAI_home')

In [50]:
def get_T_tickers():
        T_T=dbh.get(key='trained_T')
        T_T = T_T['price']
        #T_T = set(T_T)
        return T_T

In [51]:
def trained_tickers(ticker):
    T_list=[]
    T_T=dbh.get(key='trained_T')
    T_list = T_T['price']
    T_list.append(ticker)
    T_list=set(T_list)
    T_list = list(T_list)
    dbh.put({'key':'trained_T','price':T_list})


In [52]:
trained_tickers('TATAMOTORS.NS')

In [53]:
get_T_tickers()

['APOLLOHOSP.NS',
 'BAJAJFINSV.NS',
 'BAJAJ-AUTO.NS',
 'BHARTIARTL.NS',
 'BAJFINANCE.NS',
 'INDUSINDBK.NS',
 'RELIANCE.NS',
 'SBIN.NS',
 'ADANIENT.NS',
 'TATAMOTORS.NS']

In [1]:
import streamlit_lottie as st_l

2022-12-31 16:23:34.276 INFO    numexpr.utils: NumExpr defaulting to 4 threads.


In [ ]:
st_l.st_lottie()